In [ ]:
!pip install pycryptodome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.7 MB/s eta 0:00:00


In [ ]:
!pip install pycryptodome

from Crypto.Random import get_random_bytes 
from Crypto.Protocol.KDF import PBKDF2

from Crypto.Cipher import AES 
from Crypto.Util.Padding import pad, unpad 

from PIL import Image

##### DEFINING FUNCTIONS #####

def password_checker(password):
    # Check if password is at least 8 characters long
    if len(password) < 8:
        return False
    # Check if password contains at least one uppercase letter
    if not any(char.isupper() for char in password):
        return False
    # Check if password contains at least one lowercase letter
    if not any(char.islower() for char in password):
        return False
    # Check if password contains at least one digit
    if not any(char.isdigit() for char in password):
        return False
    # Check if password contains at least one special character
    special_chars = "!@#$%^&*()_+-="
    if not any(char in special_chars for char in password):
        return False
    # Password passes all checks
    return True

#key generation function. Asks user to input a password and generates a key 
def gen_key(password, salt):
  #Generate the salt
  #generate the key with user input password
  key = PBKDF2(password, salt, dkLen =32)
  return key


#ecb encryption function
def ecb_encryption(counter, mode):

      print("Enter a password that you will need later for decryption")
      print("password requirements: \n1. Minimum length is 8 characters\
      \n2. Contains at least 1 upper and lower case \n3. Contains atleast 1 digit and 1 special character (!@#$%^&*()_+-=)")
      while(1):
        password = input()
        if(password_checker(password)== True):
          break
        print("Enter a stronger password")
        
      key = gen_key(password, salt)
      #Open the image to be decrypted and convert it to bytes 
      print("Enter the location of the image you want to encrypt")
      encryption_img_location = input()
      img = Image.open(encryption_img_location)
      message = img.tobytes()

      #Create the AES cipher and configure ECB encryption mode
      cipher = AES.new(key, mode)

      #Pad the image for encryption. PKCS7 padding is used by default
      padded_image = pad(message, AES.block_size)

      #Encrypt the message
      ciphertext = cipher.encrypt(padded_image)

      #convert the image to a PIL image type
      img_out = Image.frombytes(img.mode, img.size, ciphertext)
      
      #if statement is used to name the encrypted files depending on whether 
      #ecb or cbc encrypted

      if(mode == AES.MODE_ECB):
        path = "encrypted_ecb"
        file_name = path+ str(counter)

      elif(mode == AES.MODE_CBC):
        path = "encrypted_cbc"
        file_name = path + str(counter)

      #save it as a jpeg file
      img_out.save(file_name+".jpeg")

      #Write the cipher text to a binary file. This will be used for decryption.
      with open(file_name, 'wb') as f:
        if(mode == AES.MODE_CBC):
          f.write(cipher.iv)
        f.write(ciphertext)


#decryption function 
def decrypt(mode):

      location_flag = 0

      while(location_flag ==0):
        try:
          #Open the ciphertext stored in a binary file. 
          print("Enter the location of the binary file")
          #open the encryptyed image to get mode and size information
          decryption_img_location = input()
          print("Enter the location of the jpeg file")
          img_loc = input()
          with open(decryption_img_location, 'rb') as f:
            #read the iv first if in CBC mode
            if(mode == AES.MODE_CBC):
              i_v = f.read(16)
            decrypt_data = f.read()
          location_flag = 1
        except:
          print("Please enter a valid file location")

      #Generate the key using the user input password and hard coded salt 
      print("Enter the password for decryption")
      decryption_password = input()
      key_decrypt = gen_key(decryption_password, salt)

      #opening the jpeg file to get mode and size information later
      img = Image.open(img_loc)
      
      if(mode == AES.MODE_ECB):
        #Generate the AES ECB cipher for decryption
        cipher = AES.new(key_decrypt, AES.MODE_ECB)
      if(mode == AES.MODE_CBC):
        #Generate the CBC cipher  for decryption 
        cipher = AES.new(key_decrypt, AES.MODE_CBC, iv = i_v)
      try:
        #decrypt the image and then unpad it
        original = cipher.decrypt(decrypt_data )
        original = unpad(original, AES.block_size)
        im_out = Image.frombytes(img.mode, img.size, original)
        im_out.save('decrypted' + decryption_img_location[-1] + '.jpg')
      except:
        print("Wrong password - try again")


terminate = "no"

######## SALT ##########
#this salt was generated using the following line of code in python:
#salt = get_random_bytes(32) this generates a random 32 byte (25.6 bit salt) 
salt = b'E\xfa\x1a\xb1E\x1bn\x95\xc7\xe45\xb0\xae\xde\xe1\xaeGk\x10\x0b\xeb$\x1e,\xa5\xdd.\x8d\xc6\x8fa\x0f'

#To generate a new salt that can be used for key creation:
#1. Copy the lines below and run them in a seperate script
#2. copy the new salyt generated. Delete the salt above and replace it with the new salt that you copied
#salt = get_random_bytes(32)
#print(salt) 


###### PROGRAM LOOP #######
loop_counter = 1
while(terminate != "yes"):

  #case statement to choose between encryption and decrytion
  print("MENU \nEnter 1 for ECB Encryption \nEnter 2 for CBC Encryption\
  \nEnter 3 for ECB Decryption \nEnter 4 for CBC Decryption")
  choice = input()
 
  ###### ENCRYPTION #####

  if choice == "1":
      
      ecb_encryption(loop_counter, AES.MODE_ECB)
      loop_counter = loop_counter +1
      #Ask user if they want to terminate the program 
      print("Do you want to terminate the program? Enter yes/no")
      terminate = input()
      

  elif choice == "2":

      ecb_encryption(loop_counter, AES.MODE_CBC)      
      loop_counter = loop_counter +1
      #Ask user if they want to terminate the program 
      print("Do you want to terminate the program? Enter yes/no")
      terminate = input()

  ##### Decryption #####

  elif choice == "3":

      decrypt(AES.MODE_ECB)
      #Ask user if they want to terminate the program 
      print("Do you want to terminate the program? Enter yes/no")
      terminate = input()

  elif choice == "4":

      decrypt(AES.MODE_CBC)
      #Ask user if they want to terminate the program 
      print("Do you want to terminate the program? Enter yes/no")
      terminate = input()

  else:
    print("Please enter  1, 2,3 or 4")


MENU 
Enter 1 for ECB Encryption 
Enter 2 for CBC Encryption  
Enter 3 for ECB Decryption 
Enter 4 for CBC Decryption
1
Enter a password that you will need later for decryption
password requirements: 
1. Minimum length is 8 characters      
2. Contains at least 1 upper and lower case 
3. Contains atleast 1 digit and 1 special character (!@#$%^&*()_+-=)
g
Enter a stronger password
ghj
Enter a stronger password
Hell01!234
Enter the location of the image you want to encrypt
ntent/tux.jpg


FileNotFoundError: ignored